# Single Bird Flight analysis


In [1]:
%matplotlib inline

from pathlib import Path

#from yupiwrap.tracktable import yupi2tracktable
#from tracktable.core.geomath import length
import matplotlib.pyplot as plt
import numpy as np
#from tracktable.core.geomath import length, end_to_end_distance
from yupi.graphics import plot_velocity_hist
from yupi.stats import speed_ensemble

from birdwatchpy.sequences.sequences_helper import load_sequence_from_pickle
from birdwatchpy.bird_flight_analysis.trajs_plot import plot_2D


In [2]:
# parameter (this parameter will be injected by papermill)
sequence_dir_path = ""
flight_id = ""
byte_pickle = b''

In [3]:
sequence = load_sequence_from_pickle(Path(f"{sequence_dir_path}/{Path(sequence_dir_path).name}.sequence"))
single_flight = list(filter(lambda flight: flight.flight_id == flight_id, sequence.birds)).pop()

FileNotFoundError: [Errno 2] No such file or directory: '/.sequence'

### Sequence Video

In [ ]:
from IPython.display import Video
video_path=f"{sequence_dir_path}/{sequence.sequence_id}.webm"
if Path(video_path).is_file():
    Video(video_path)
    print(video_path)
else:
    print(f"webm video not found: {video_path}")

### Plot Trajectory

In [ ]:

plot_2D(single_flight.y_traj, legend=False)

### Basic Data

In [ ]:
print(single_flight.get_basic_table())

### Straightness Tables

In [ ]:
for straightness_table in single_flight.get_straightness_tables():
    print(straightness_table)

### Bird size

In [ ]:
# Number of samples
N = len(single_flight.sizes)

x = np.linspace(0, len(single_flight.sizes) * 1 / 30, len(single_flight.sizes))
plt.xlabel('seconds')
plt.ylabel('bb area')
plt.plot(x, single_flight.sizes, x, np.repeat(single_flight.avg_size, len(single_flight.sizes)), '--')

In [ ]:
x = np.linspace(0, len(single_flight.sizes) * 1 / 30, len(single_flight.sizes))

heights = [bb.height for bb in single_flight.bounding_boxes]
widths = [bb.width for bb in single_flight.bounding_boxes]
avg_height = np.repeat(np.average(heights), len(single_flight.sizes))
avg_width = np.repeat(np.average(widths), len(single_flight.sizes))
plt.plot(x, heights, "g", label="height")
plt.plot(x, avg_height, 'g--', label="avg_height")
plt.plot(x, widths, "r", label="width")
plt.plot(x, avg_width, 'r--', label="avg_width")

plt.legend()
plt.title("Bounding box sizes over time", fontsize=15)

plt.xlabel('seconds')
plt.ylabel('pixel')


### Wing Flap Frequency

In [ ]:
from scipy.fft import fft, fftfreq
from matplotlib.pyplot import figure


_, xf, yf = single_flight.calc_wing_flap_freq()

figure(figsize=(8, 6), dpi=100)
plt.xlabel('flaps per second')
plt.ylabel('Frequency Domain Magnitude')
plt.plot(xf, np.abs(yf), )
plt.ticklabel_format(useOffset=False)
plt.show()



print("--- Naive sliding window ---")

a = np.abs(yf)[2:]
n=len(a)
k=4
ms=-10**6
ws=sum(a[:k])
res = []
for i in range(n-k):
    if ws>ms:
        pos = i
        res.append(xf[2:][pos])
    ms=max(ms,ws)
    ws=ws-a[i]+a[i+k]
ms=max(ms, ws)
#print(pos)
#print(xf[2:][pos])
print(f"Frequency: {res[-1]}")
print("Top candidates:")
print(res)


### Histogram of Velocity

In [ ]:

v = speed_ensemble([single_flight.y_traj], step=1)
plot_velocity_hist(v, bins=5)

### Histogram of Turning Angles